<a href="https://colab.research.google.com/github/Kranthi98/Udemy-python-ML/blob/master/lectures/13_may_udemy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Recommendations
#####learning the movie recommendations 

---



In [0]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Input, Dropout, Flatten, Add, Dense, Dot, Embedding
from keras.models import Model
from keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [0]:

!wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip

--2020-05-24 11:11:47--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K   812KB/s    in 1.2s    

2020-05-24 11:11:49 (812 KB/s) - ‘ml-latest-small.zip’ saved [978202/978202]



In [0]:
!unzip ml-latest-small.zip

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [0]:

movies = pd.read_csv('ml-latest-small/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [0]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [0]:
len(movies.movieId.unique())

9742

In [0]:
len(ratings.userId.unique())

610

###Collabarotive filtering
####it is straight forward and effective method for high quality recommendations
###How it works??
####it optimises both the movie and users weights and then gets the dot product of both the vectors and get the predictons and then optimises the weights by comparing with the original weights

##Merge function
###data = data1.merge(data2,on = "column on which merge shud be done : foreign key in SQL").drop(["list of columns that shud be dropped"],axis = 1)

In [0]:
ratings = ratings.merge(movies,on="movieId").drop(["timestamp","genres"],axis = 1)

In [0]:
ratings.head()

,userId,movieId,rating,title,user_seq,item_seq,movie_seq
0,1,1,4.0,Toy Story (1995),0,0,0
1,5,1,4.0,Toy Story (1995),4,0,0
2,7,1,4.5,Toy Story (1995),6,0,0
3,15,1,2.5,Toy Story (1995),14,0,0
4,17,1,4.5,Toy Story (1995),16,0,0


##Generate Sequential identifiers
####The model needs sequential numbers...but the moviesID and userID have arbitrary numbers..So to generate the sequential numbers we use label encoder from scikit learn..

In [0]:
from sklearn.preprocessing import LabelEncoder


In [0]:
user_seq = LabelEncoder()
ratings['user_seq'] = user_seq.fit_transform(ratings['userId'].values)
iten_seq = LabelEncoder()
ratings['movie_seq'] = iten_seq.fit_transform(ratings['movieId'].values)

In [0]:
del ratings['item_seq']

In [0]:
ratings.head()

,userId,movieId,rating,title,user_seq,movie_seq
0,1,1,4.0,Toy Story (1995),0,0
1,5,1,4.0,Toy Story (1995),4,0
2,7,1,4.5,Toy Story (1995),6,0
3,15,1,2.5,Toy Story (1995),14,0
4,17,1,4.5,Toy Story (1995),16,0


##splitting the data 
####here its a bit unusual its not the general train-test split..test data will be same which is equal to part of data i.e., either 20 percent or 30 percent or 40 percent
####But the thing is for training we gonna use all the data..Bcoz excluding the data might remove all the data of certain users or movies



In [0]:
train_unused, test = train_test_split(ratings,test_size = 0.2, random_state = 0)
train = ratings

###Keras Embedding Layer
####the model produces set of vectors from numbers
####for example there will be movie names or words which we have to choose in a ML model..Embedding layer does this,by converting each object into set of vector


*   no od total values being fed called as input dimension
*   output dimension where we give set of no of output numbers..,
####higher the numbers higher will be the dimensionality and we can arrange the moral complex things..,so does it changes the computation time


###Model preparation

In [0]:
num_features = 50
dropout = 0.0